In [8]:
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import cv2 

TRAIN_DIR = 'C:/Users/alejo/tesis/train'
TEST_DIR = 'C:/Users/alejo/tesis/test'
#tamaño de imagen
IMG_SIZE = 50
#learning rate
LR = 1e-3
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

In [9]:
def labelImg(img):
    #dog.90.png
    wordLabel = img.split('.')[-3]
    if wordLabel == 'cat': return [1,0]
    elif wordLabel == 'dog': return [0,1]

In [10]:
def createTrainData():
    trainingData = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = labelImg(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        trainingData.append([np.array(img), np.array(label)])
    shuffle(trainingData)
    np.save('trainData.npy',trainingData)
    return trainingData

In [11]:
def processTestData():
    testingData = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        imgNum = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testingData.append([np.array(img),imgNum])
    np.save('testData.npy',testingData)
    return testingData

In [12]:
trainData = createTrainData();
#trainData = np.load('trainData.npy')

100%|███████████████████████████████████████████████████████████████████████████| 24999/24999 [00:45<00:00, 546.48it/s]


In [13]:
testData = processTestData();
#testData = np.load('testData.npy');

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:31<00:00, 391.98it/s]


In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

#reinicia la grafica que sale en el tensorboard
import tensorflow as tf
tf.reset_default_graph()

#usando tf learn
#capa de entrada
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')
## 2 capas de convolución y pooling
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

## se agrega una capa completamente conectada
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
##capa de salida
#opciones que se tienen para reconocer 2 gato y perro, segundo arg
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
#creación del modelo
model = tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded')
else:
    print('fail')

In [ ]:
train = trainData[:-500]
test = trainData[-500:]

In [ ]:
#x feature sets, Y Labels
#i[0] data de la imagen pixel data
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

testX = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testY = [i[1] for i in test]

In [ ]:
#entrenamiento del modelo
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': testX}, {'targets': testY}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [ ]:
#tensorboard --logdir=foo:C:\Users\alejo\tesis\log
model.save(MODEL_NAME)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()

for num, data in enumerate(testData[:12]):
    #cat[1,0]
    #dog[0,1]
    imgNum = data[1]
    imgData = data[0]
    
    y = fig.add_subplot(3,4,num+1)
    orig = imgData
    data = imgData.reshape(IMG_SIZE,IMG_SIZE,1)
    
    modelOut = model.predict([data])[0]
    
    if np.argmax(modelOut) == 1:str_label='Dog'
    else str_label='Cat'
    
    y.imshow(orig, cmap='gray')
    
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(false)
    y.axes.get_yaxis().set_visible(false)
plt.show()

In [ ]:
#model accuracy
model.evaluate(x_test, y_test)